<a href="https://colab.research.google.com/github/skmelo/MVP_MachineLearning/blob/main/MVP_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>